In [1]:
from pyspark.sql import SparkSession

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
import  pyspark.sql.functions as F

import datetime

import numpy as np

import os
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1635248200664_0014,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession \
            .builder \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
            .appName("Taxi Data") \
            .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
baseUrl= "s3://taxi-cab-spark-temp/"
taxi_data = baseUrl + "raw/*.csv"
out_data = baseUrl + "output/yellow_taxi.parquet"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
 # Read taxi data file
print("Reading taxi CSV files from S3")
df = spark.read.option("header",True).csv(taxi_data, mode='PERMISSIVE', \
                         columnNameOfCorruptRecord='corrupt_record').dropna().dropDuplicates()
print("Read completed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading taxi CSV files from S3
Read completed

In [ ]:
df.printSchema()
df.describe("trip_distance","fare_amount","total_amount").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import  pyspark.sql.functions as F
dfYellow = df \
                .withColumn("VendorID", df["VendorID"].cast(IntegerType())) \
                .withColumn("tpep_dropoff_datetime", F.to_timestamp("tpep_dropoff_datetime")) \
                .withColumn("tpep_pickup_datetime", F.to_timestamp("tpep_pickup_datetime")) \
                .withColumn("passenger_count", df["passenger_count"].cast(IntegerType())) \
                .withColumn("trip_distance", df["trip_distance"].cast(FloatType())) \
                .withColumn("RatecodeID", df["RatecodeID"].cast(IntegerType())) \
                .withColumn("payment_type", df["Payment_type"].cast(IntegerType())) \
                .withColumn("fare_amount", df["Fare_amount"].cast(FloatType())) \
                .withColumn("extra", df["extra"].cast(FloatType())) \
                .withColumn("mta_tax", df["mta_tax"].cast(FloatType())) \
                .withColumn("tip_amount", df["tip_amount"].cast(FloatType())) \
                .withColumn("improvement_surcharge", df["improvement_surcharge"].cast(FloatType())) \
                .withColumn("PULocationID", df["PULocationID"].cast(IntegerType())) \
                .withColumn("DOLocationID", df["DOLocationID"].cast(IntegerType())) \
                .withColumn("total_amount", df["total_amount"].cast(FloatType())) \
                .withColumn("congestion_surcharge", df["congestion_surcharge"].cast(FloatType()))


dfYellow.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)

In [7]:
dfYellow.describe("trip_distance","fare_amount","total_amount").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+------------------+
|summary|     trip_distance|       fare_amount|      total_amount|
+-------+------------------+------------------+------------------+
|  count|          23838930|          23838930|          23838930|
|   mean|2.7470826823464476|12.134632555010477|17.918562511342326|
| stddev|44.072016766632146| 278.6755982610255| 345.9320460785234|
|    min|            -22.18|           -1259.0|           -1260.3|
|    max|         167329.45|          998310.0|         1000003.8|
+-------+------------------+------------------+------------------+

In [8]:
dfYellow = dfYellow.filter((dfYellow['trip_distance']>0.1) & (dfYellow['tip_amount']>=0) & (dfYellow['total_amount']>0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
dfYellow.describe("trip_distance","fare_amount","tip_amount","total_amount").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
dfYellow.count()

Save to Parquet

In [ ]:
 dfYellow.write.mode("overwrite").parquet(out_data)

In [ ]:
parDF=spark.read.parquet(out_data)

In [ ]:
parDF.printSchema()


In [ ]:
parYellowDF = parDF \
                .withColumn("hour", F.hour("tpep_pickup_datetime")) \
                .withColumn("month", F.month("tpep_pickup_datetime")) \
                .withColumn("year", F.year("tpep_pickup_datetime")) \
                .withColumn("quarter", F.quarter("tpep_pickup_datetime")) \
                .withColumn("month", F.month("tpep_pickup_datetime")) \
                .withColumn("trip_duration", (F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long"))/60.)

parYellowDF.printSchema()
parYellowDF.describe("trip_distance","fare_amount","total_amount","tpep_dropoff_datetime","tpep_pickup_datetime","trip_duration").show()


In [13]:
parYellowDF = parYellowDF.filter((parYellowDF['trip_duration']>0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
get_speed = udf(lambda x,y : x/y, FloatType())
get_tip_proportion = udf(lambda x,y : x/(y/60), FloatType())

parYellowDF = parYellowDF \
                        .withColumn("tip_pro", get_tip_proportion('tip_amount','total_amount')) \
                        .withColumn("speed", get_tip_proportion('trip_distance','trip_duration'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
parYellowDF.createOrReplaceTempView("rides")
spark.sql("""
    SELECT t.quarter, t.DOlocationID, r.max_tip_prop
    FROM (
          SELECT DISTINCT quarter , max(tip_pro) as max_tip_prop
        FROM rides
        group by quarter
    ) r
    INNER JOIN rides t
    ON t.quarter = r.quarter AND t.tip_pro = r.max_tip_prop 
    ORDER BY 1
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+------------+
|quarter|DOlocationID|max_tip_prop|
+-------+------------+------------+
|      1|         138|        60.0|
|      2|          23|    59.78898|
|      3|         265|   59.953884|
|      4|         265|   59.869373|
+-------+------------+------------+

In [16]:
parYellowDF.createOrReplaceTempView("rides")
spark.sql("""
    SELECT DISTINCT hour , max(speed) as max_speed
    FROM rides
    group by hour
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+
|hour|max_speed|
+----+---------+
|  12|  35640.0|
|   1|  14940.0|
|   6|21155.998|
|   3|  11400.0|
|   4|   7200.0|
|   8|  12240.0|
|  11|  39120.0|
|  19|  26820.0|
|  23|31139.998|
|  21|  44730.0|
|  14|  34020.0|
|  16|  20880.0|
|  20|  31320.0|
|   5|  12600.0|
|  15|58679.996|
|   2|  16200.0|
|  18|  34380.0|
|  13|  23040.0|
|  17|  42120.0|
|   7|  33840.0|
+----+---------+
only showing top 20 rows

In [23]:
parYellowDF.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+----+-----+----+-------+-----------------+---------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|hour|month|year|quarter|    trip_duration|  tip_pro|    speed|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+----+-----+----+-------+-----------------+---------+---------+
|       2| 2020-01-01 00:12:40|  2020